In [1]:
from datetime import datetime, timedelta
from IPython.display import display, HTML
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.dataframe import DataFrame
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [2]:
CRYPTOS = ['BTC', 'VEN', 'ETH', 'EOS', 'LTC', 'XRP', 'BCH', 'XLM', 'CVC']

In [31]:
spark = SparkSession.builder.appName("Crypto Data").getOrCreate()
spark.sparkContext.setCheckpointDir('/tmp/checkpoint/')
spark.sparkContext.setLogLevel('INFO')
df = spark.read.csv("/home/jovyan/data/data_train_test.csv", inferSchema=True, encoding='utf-8', header=True).cache()

In [35]:
df = df.withColumn("year", F.year(df['time']))

In [48]:
df = df.withColumn("6h growth", (df['price-0h']/df['price-6h'])-1)

In [49]:
df = df.withColumn("6h round", F.round(df['6h growth'],1))

In [45]:
df.filter(df['time'] > datetime(2018,5,31)).select('time', '6h growth').orderBy('time').show()

+-------------------+--------------------+
|               time|           6h growth|
+-------------------+--------------------+
|2018-05-31 01:00:00|0.030103879584481463|
|2018-05-31 01:00:00| 0.03178323645227832|
|2018-05-31 01:00:00|0.020166777790970647|
|2018-05-31 01:00:00| 0.06232008091496177|
|2018-05-31 01:00:00| 0.01488209458018841|
|2018-05-31 01:00:00|0.001006833615389...|
|2018-05-31 01:00:00|0.025658393702434834|
|2018-05-31 01:00:00|0.002265861027190...|
|2018-05-31 02:00:00|0.011301485264003652|
|2018-05-31 02:00:00|0.006972492666128094|
|2018-05-31 02:00:00|0.017534042156314156|
|2018-05-31 02:00:00|-0.00348298040556...|
|2018-05-31 02:00:00| 0.05912397461693231|
|2018-05-31 02:00:00|0.020507399577166963|
|2018-05-31 02:00:00|0.013906125696101945|
|2018-05-31 02:00:00|-7.51879699248059...|
|2018-05-31 03:00:00| 0.01576263328697247|
|2018-05-31 03:00:00| 0.01583733814773236|
|2018-05-31 03:00:00|0.013145020632152171|
|2018-05-31 03:00:00| 0.09083838855187443|
+----------

In [62]:
df.groupby('year').agg(F.countDistinct('6h round')).show()

+----+------------------------+
|year|count(DISTINCT 6h round)|
+----+------------------------+
|2018|                       9|
|2015|                      25|
|2013|                      14|
|2014|                      25|
|2016|                      24|
|2017|                      27|
+----+------------------------+



In [73]:
df.groupby('symbol', 'year', '6h round').agg({'6h round':'count'}).orderBy('year').filter(df['year'] > 2016).toPandas().head(50)

,symbol,year,6h round,count(6h round)
0,XLM,2017,0.6,2
1,BCH,2017,0.4,5
2,VEN,2017,44487.6,1
3,XRP,2017,-0.2,32
4,XLM,2017,1.1,1
5,VEN,2017,43175.2,1
6,ETH,2017,-0.1,348
7,BCH,2017,0.2,44
8,CVC,2017,-1.0,1
9,VEN,2017,15188.0,1


In [66]:
df.filter(df['6h round'] == 15188.0).toPandas().head()

,_c0,time,symbol,price-0h,price-1h,price-2h,price-4h,price-5h,price-6h,price-8h,...,btc-vol-24h,btc-vol-48h,btc-vol-96h,btc-vol-192h,btc-vol-384h,btc-vol-768h,label,6h growth,year,6h round
0,71827,2017-10-04 15:00:00,VEN,5146.03095,0.3112,0.320075,0.340825,0.34295,0.3388,0.358825,...,11136824.04,16191066.52,20994487.49,13140344.09,32051604.59,46320892.15,1,15187.993359,2017,15188.0
